<a href="https://colab.research.google.com/github/Xenov-X/penglab/blob/master/Restore%7C%7CContinue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Penglab (Abuse of Google Colab for fun and profit)
#by mxrch

#Choose what you want to install
hashcat = True
john = False
hydra = False


#Map google drive?
map_GDrive = True #Will map a folder called "hashcat" in root of google drive to /root/.hashcat


#Which shell ? (only one, or none)
#⚠️ Don't select a shell to run bash commands in the Google Colab way (you'll see). Stable.
ssh = False #It uses ngrok. Stable and interactive, but take a little time to setup
python_shell = False #Useful for little hashes, but not for long hours, since you'll see the output only when it finish. Not interactive, but stable.
bash_shell = False #Useful for big hashes. but you'll don't see your input (surely a Google Colab protection). Stable.



##OR USE GDRIVE ABOVE TO STORE WORDLISTS
#Wordlists (see weakpass.com to download them)
wordlists_dir = "wordlists"

rockyou = False #For CTFs, especially Hack The Box (133.44 Mb)
hashesorg2019 = False #Very heavy (12.79 Gb) but has a good rate, if you're determinated

#---------------------------

if (python_shell and bash_shell) or (bash_shell and ssh) or (ssh and python_shell) :
    print("Please do a choice")
    exit()

In [2]:
# Get hashes to crack
# Make a directory called 'dothashcat' in your Google Drive before this
# with the 'hashes' subdirectory for storing hashes
if map_GDrive:
  from google.colab import drive

# This will ask you to go to a link and get an authorization code
# to let the .hashcat folder get synced to your Google Drive
  drive.mount('/content/drive')

# Make a symbolic link between Google Drive
# and the local .hashcat directory
  !ls /content/drive/My\ Drive/hashcat
  !ln -s /content/drive/My\ Drive/hashcat /root/.hashcat


  !printf "[+] \n\nGoogle Drive mapped at /root/.hashcat!\n[+] ls /root/.hashcat\n"
  !ls /root/.hashcat/

Mounted at /content/drive
hashcat		   hashcat.potfile  kernels   wordlists
hashcat.dictstat2  hashes	    sessions
[+] 

Google Drive mapped at /root/.hashcat!
[+] ls /root/.hashcat
hashcat		   hashcat.potfile  kernels   wordlists
hashcat.dictstat2  hashes	    sessions


In [3]:
#Wordlists
import os

os.system("wordlists_dir={}".format(wordlists_dir))
!mkdir ./$wordlists_dir
if rockyou:
    !printf "[+] Downloading the Rockyou wordlist...\n"
    !cd $wordlists_dir && wget https://download.weakpass.com/wordlists/90/rockyou.txt.gz
    !printf "[+] Wordlist downloaded !\n[+] Extraction...\n"
    !cd $wordlists_dir && gunzip rockyou.txt.gz
    !printf "[+] Finished !\n[+] Location : $(pwd)/$wordlists_dir/$(ls wordlists | grep rockyou)"

if hashesorg2019:
    !printf "[+] Downloading the HashesOrg2019 wordlist...\n"
    !cd $wordlists_dir && wget https://download.weakpass.com/wordlists/1851/hashesorg2019.gz
    !printf "[+] Wordlist downloaded !\n[+] Extraction...\n"
    !cd $wordlists_dir && gunzip hashesorg2019.gz
    !printf "[+] Finished !\n[+] Location : $(pwd)/$wordlists_dir/$(ls wordlists | grep hashesorg2019)"

rockyou.txt.gz        0%[                    ] 485.85K  4.83KB/s    eta 3h 47m ^C
[+] Wordlist downloaded !
[+] Extraction...

gzip: rockyou.txt.gz: unexpected end of file
[+] Finished !
[+] Location : /content/wordlists/rockyou.txt.gz

In [ ]:
#Install of hashcat
if hashcat:
    print("Installation of hashcat...")
    !apt install cmake build-essential -y
    !apt install checkinstall git -y
    !git clone https://github.com/hashcat/hashcat.git && cd hashcat && make -j 8 && make install

In [5]:
#Install of john
if john:
    print("Installation of john...")
    !apt-get install john -y

In [6]:
#Install of hydra
if hydra:
    print("Installation of hydra...")
    !apt install cmake build-essential -y
    !apt install checkinstall git -y
    !git clone https://github.com/vanhauser-thc/thc-hydra.git && cd thc-hydra && ./configure && make && make install

In [ ]:
#Setting up shells
from termcolor import colored
import os

if ssh:
    print("Setting up SSH...")
    !pip install git+https://github.com/demotomohiro/remocolab.git
    import remocolab
    remocolab.setupSSHD()

elif python_shell:
    os.system("touch /tmp/cmd && touch /tmp/status")

    template_cmd = "echo -n $(whoami)[$(hostname)[$(pwd) &> /tmp/status"
    os.system("bash -c '{}'".format(template_cmd))
    output = {"cmd": "", "status": ""}
    with open('/tmp/cmd', 'r') as f:
        output["cmd"] = f.read()
    with open('/tmp/status', 'r') as f:
        output["status"] = f.read()
    prefixes = output["status"].split("[")
    path = prefixes[2].replace('\n', '')
    prefix = colored(prefixes[0] + "@" + prefixes[1], "red") + ":" + colored(path, "cyan") + "$ "
    print("")

    while 1:
        print(prefix, end='')
        cmd = input()
        template_cmd = "cd {} && ".format(path) + "" + cmd + " &> /tmp/cmd ; echo $(whoami)[$(hostname)[$(pwd) &> /tmp/status"
        os.system("bash -c '{}'".format(template_cmd))
        output = {"cmd": "", "status": ""}
        with open('/tmp/cmd', 'r') as f:
            output["cmd"] = f.read()
        with open('/tmp/status', 'r') as f:
            output["status"] = f.read()
        prefixes = output["status"].split("[")
        path = prefixes[2].replace('\n', '')
        prefix = colored(prefixes[0] + "@" + prefixes[1], "red") + ":" + colored(path, "cyan") + "$ "
        print(output["cmd"])

elif bash_shell:
    !/bin/bash

else:
    print('\n🚨🚨🚨\n🎉 Your environment is ready !\nWant to run it from the code blocks ?')
    print('Just use the code block below and start your commands by a "!" (make sure to uncomment them).')
    print('▶️ When you\'re ready, hit the run button at the left of the code block !\n')
    print('|\n|\n|\nV')

In [ ]:
# Put your code here :
# !hashcat -h
# !john --help!
!hashcat --restore





